# 110. Content Based Filtering - manual feature engineering

- 사용자, 영화 및 feature 목록을 만듭니다. 사용자와 영화는 데이터베이스에서 가져왔다고 가정  

- 영화의 feature 는 수작업으로 작성되었으며 도메인 지식에 의존하여 최상의 임베딩 공간을 제공. 여기에서는 액션, 공상 과학, 코미디, 만화 및 드라마 카테고리를 사용하여 영화(및 사용자)를 설명합니다.

- 이 예에서는 데이터베이스가 아래에 나열된 4 명의 사용자와 6 개의 영화로 구성되어 있다고 가정합니다.

## Content-Based Recommender System algorithm

Step 1 : 사용자의 영화 평가를 근거로 weighted genre matrix 작성 (Genre 별로 평점 합산 및 normalize)  
Step 2 : Weighted genre matrix 로부터 User Profile 작성  
Step 3 : 사용자가 보지 않은 Movie matrix 에 User Feature Profile 을 적용하여 추천 영화 list 작성

## 사용자, 영화 등급 (movie ratings) 및 특성(feature) 초기화

- 사용자의 영화 등급과 k-hot-encoding 된 영화 특성 매트릭스를 입력해야합니다.   
- users_movies matrix의 각 행은 각 영화에 대한 단일 사용자 등급 (1 ~ 10)을 나타냅니다. 0은 사용자가 해당 영화를 보거나 평가하지 않았음을 나타냅니다.   
- movies_feats matrix에는 주어진 영화 각각에 대한 feature가 포함되어 있습니다. 각 행은 6 개의 영화 중 하나를 나타내고 열은 5 개의 카테고리를 나타냅니다. 1 은 영화가 특정 장르/카테고리에 해당함을 나타냅니다.

## step1) 사용자 특성 매트릭스 (weighted genre matrix) 계산

사용자의 영화 평가를 근거로 weighted genre matrix 작성 (Genre 별로 평점 합산 및 normalize)

즉, 5 차원 특성 공간에 각 사용자의 임베딩을 포함하는 행렬입니다.  

이것을 `users_movies` matrix와 `movies_features` matrix의 행렬 곱셈으로 계산합니다.

## step2) Weighted genre matrix 로부터 User Profile 작성
다음으로 각 사용자 특성 벡터를 정규화하여 1 이 되도록 정규화합니다. 

정규화는 꼭 필요한 것은 아니지만 사용자간에 등급 규모를 비교할 수 있도록 합니다.

## step3) 사용자가 보지 않은 Movie matrix 에 User Feature Profile 을 적용하여 추천 영화 list 작성

위에서 계산된 users_feats 를 사용하여 각 사용자에 대한 각 영화 카테고리의 상대적 중요성을 나타낼 수 있습니다.

```
tf.nn.top_k : 마지막 차원에 대한 'k' 가장 큰 항목의 값과 인덱스를 찾습니다.

result = tf.math.top_k([1, 2, 98, 1, 1, 99, 3, 1, 3, 96, 4, 1], k=3)
>>> result.values.numpy()
array([99, 98, 96], dtype=int32)
>>> result.indices.numpy()
array([5, 2, 9], dtype=int32)
```

각 사용자별로 선호 장르를 순서대로 나열

장르 index 를 장르명으로 표시

## 추천할 영화 결정

- 이제 위에서 계산 한 `users_features` 를 사용하여 각 사용자에 대한 영화 등급 및 추천을 결정합니다.  
- 각 영화의 예상 등급을 계산하기 위해 사용자의 특성 벡터와 해당 영화 특성 벡터간의 유사성 측정값을 계산합니다.  

### 모든 영화 특성과 모든 사용자 특성간의 유사도 측정

- 위의 계산은 데이터베이스의 각 사용자와 각 영화 간의 유사성 측정값을 계산합니다.  
- 새 영화의 등급에만 집중하기 위해 all_users_ratings 매트릭스에 마스크를 적용하여, 사용자가 이미 영화를 평가한 경우 해당 평가를 무시합니다. 이렇게 하면 이전에 본 적이 없거나 등급이 지정되지 않은 영화에 대한 등급에만 집중합니다.

- 각 사용자에 대해 보지 않은 영화의 상위 2 개 등급의 영화를 가져와 인쇄해 봅니다.

추천할 영화